In [1]:
from util_dev import *


AWS SETUP
FILTER ENGINE SETUP
VECTOR SEARCH SETUP
LEXICAL SETUP
SEMANTIC SETUP
LLM SETUP


In [4]:
series2doc['800 (80B)']

['Arguments_Example_800']

# Single Test

In [25]:
# query = 'Is the 700 series user friendly?'
# filters = {"series": ['700 Gen7'],
#             "models": [],
#             "languages": ["English"]}



# query = 'Tell me about the agility on the 800'
# filters = {"series": ['800'],
#             "models": [],
#             "languages": ["English"]}


query = 'Does the 724 have Dynamic Performance?'
filters = {"series": ['700 Gen7'],
            "models": [],
            "languages": ["English"]}


In [26]:
out = ""
for r in make_resp(query, filters):
    out += r

print(out)


QUERY: Does the 724 have Dynamic Performance?
FETCHING TEXT CONTENT
txts/2024-profiSonderdruck-728VarioGen7_1.txt
txts/Product_presentation_700_Vario_Gen7_31.txt
txts/Product_presentation_700_Vario_Gen7_28.txt
txts/Product_presentation_700_Vario_Gen7_30.txt
txts/Product_presentation_700_Vario_Gen7_32.txt
txts/Cab_700_Vario_Gen7_24.txt
txts/Product_presentation_700_Vario_Gen7_60.txt
txts/Product_presentation_700_Vario_Gen7_33.txt
txts/Product_presentation_700_Vario_Gen7_127.txt
txts/Product_presentation_700_Vario_Gen7_29.txt
txts/Product_presentation_700_Vario_Gen7_5.txt
txts/Product_presentation_700_Vario_Gen7_16.txt
txts/Product_presentation_700_Vario_Gen7_4.txt
txts/Product_presentation_700_Vario_Gen7_9.txt
txts/Product_presentation_700_Vario_Gen7_11.txt
txts/Product_presentation_700_Vario_Gen7_84.txt
txts/Product_presentation_700_Vario_Gen7_13.txt
txts/2024-profiSonderdruck-728VarioGen7_2.txt
txts/2024-profiSonderdruck-728VarioGen7_7.txt
txts/2024-profiSonderdruck-728VarioGen7_4.txt

# Test Table

In [ ]:
# Load the test questions
df = pd.read_csv(INPUT_DIR + "/test_questions.csv")
# df = df.head(25)
questions = df.to_dict(orient="records")
df

In [ ]:
filters

In [ ]:
# Ask Gpt
for row in questions:
    query = row["English"]
    filters = {"series": [row["Series"]] if type(row["Series"]) == str else [],
                "models": [row["Model"]] if type(row["Model"]) == str else [],
                "languages": ["English"]}
    out = ""
    for r in make_resp(query, filters):
        out += r
    row["Response"] = out

In [ ]:
# Extract sources
for row in questions:
    sources = []

    for match in re.finditer(r'\[([^\]]+)\]', row["Response"]):
        sources.append(int(match.group(0)[1:-1]))
    row["Sources"] = list(set(sources))

for row in questions:
    source_tuples = []
    for source_id in row["Sources"]:
        source_tuples.append((source_id, id2doc[source_id]))
    row["Sources"] = source_tuples
    
questions

In [ ]:
df_out = pd.DataFrame(questions)
df_out.to_csv(OUTPUT_DIR + "/test_responses_3.csv", index=False)
df_out.to_excel(OUTPUT_DIR + "/test_responses_3.xlsx", index=False)

In [ ]:
df_out = pd.read_csv(OUTPUT_DIR + "/test_responses.csv")

# See Feedback

In [1]:
import pandas as pd

import pickle, boto3, botocore
from dotenv import load_dotenv
load_dotenv() #for local development
#AWS init
aws_session = boto3.Session()
s3 = aws_session.client(
    's3', 
    config = botocore.client.Config(
        s3={'use_accelerate_endpoint': True},
        max_pool_connections=10000)
)
aws_account_num = boto3.client('sts').get_caller_identity().get('Account')
S3_BUCKET = f"sales-and-marketing-{aws_account_num}"

def get_text(key):
    response = s3.get_object(Bucket=S3_BUCKET, Key=key)
    content = response['Body'].read().decode('utf-8')
    return content

def get_aws_file(key):
    obj = s3.get_object(Bucket=S3_BUCKET, Key=key)
    data = pickle.loads(obj['Body'].read())
    return data

In [4]:
obj = s3.get_object(Bucket=S3_BUCKET, Key="feedback.csv")
df = pd.read_csv(obj["Body"])

In [5]:
df

,rating,comment,query,response,filters,sources,time
0,Very Helpful,This is a test comment,Test question,This is a test response,"{'group': 'All Groups', 'model': 'All Models',...","[{'doc_name': 'Test Document', 'page_num': 1, ...",2025-03-28 08:37:21.426131
1,Somewhat Helpful,test,How does the Fendt iD Low-Speed-Concept differ...,The Fendt iD Low-Speed-Concept differs from ot...,"{'group': 'All Product Groups', 'series': 'All...",[{'doc_name': 'Product_presentation_700_Vario_...,2025-03-28 08:41:03.510584
2,Somewhat Helpful,test\n,How does the Fendt iD Low-Speed-Concept differ...,The Fendt iD Low-Speed-Concept differs from ot...,"{'group': 'All Product Groups', 'series': 'All...",[{'doc_name': '592541_2201-PL797-B2B_Vorstellu...,2025-03-31 15:33:15.676281
3,Not Helpful,testing,What are the technical specifications of the F...,The technical specifications of the Fendt 724 ...,"{'group': 'All Product Groups', 'series': 'All...",[{'doc_name': 'Product_presentation_700_Vario_...,2025-03-31 09:49:39.187292
4,Somewhat Helpful,test,How does the Fendt iD Low-Speed-Concept differ...,The Fendt iD Low-Speed-Concept differs from ot...,"{'group': 'All Product Groups', 'series': 'All...",[{'doc_name': '697755_Product_presentation_900...,2025-03-31 18:41:50.822844
